<a href="https://colab.research.google.com/github/NMPau/Project/blob/main/PDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import DivIcon
import warnings
import scipy as sp
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import seaborn as sns
sns.set_style('white')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd drive/My\ Drive

/content/drive/My Drive


In [4]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [5]:
cd DA

/content/drive/My Drive/Colab Notebooks/DA


In [ ]:
!ls

airlines.csv  flights.csv    L_AIRPORT_ID.csv
airports.csv  L_AIRPORT.csv  PDA.ipynb


In [6]:
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv', low_memory = False)
aircode1 = pd.read_csv('L_AIRPORT.csv')
aircode2 = pd.read_csv('L_AIRPORT_ID.csv')

Проверка дубликатов

In [ ]:
print(f'Duplicates in df: {flights.iloc[:,1:].duplicated().sum()}')

Duplicates in df: 0


Форматирование кодов аэропортов

In [7]:
aircode1 = aircode1.reset_index()
aircode2 = aircode2.reset_index()
aircodes = pd.merge(aircode1,aircode2,on='Description')
aircode_dict = dict(zip(aircodes['Code_y'].astype(str),aircodes['Code_x']))

In [8]:
flights['ORIGIN_AIRPORT'] = flights['ORIGIN_AIRPORT'].values.astype(str)
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

for i in range(len(flights)):
    if len(flights['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = flights['ORIGIN_AIRPORT'][i]
        value = aircode_dict[flights['ORIGIN_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)
for i in range(len(flights)):
    if len(flights['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = flights['DESTINATION_AIRPORT'][i]
        value = aircode_dict[flights['DESTINATION_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

изменение формата даты

In [9]:
month_dict={
    1:  'January',
    2:  'February',
    3:  'March',
    4:  'April',
    5:  'May',
    6:  'June',
    7:  'July',
    8:  'August',
    9:  'September',
    10: 'October',
    11: 'November',
    12: 'December'
}
flights['MONTH_desc'] = flights['MONTH'].apply(lambda m: month_dict[m])

In [10]:
day_of_week_dict = {
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday',
    7: 'Monday'
}
flights['DOW_desc'] = flights['DAY_OF_WEEK'].apply(lambda d: day_of_week_dict[d])

In [15]:

flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH','DAY']], yearfirst=True)

формат времени

In [16]:
import datetime
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

In [17]:
def combine_date_heure(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_heure(cols))
        else:
            cols[1] = format_heure(cols[1])
            liste.append(combine_date_heure(cols))
    return pd.Series(liste)

In [20]:
flights['SCHEDULED_DEPARTURE'] = create_flight_time(flights, 'SCHEDULED_DEPARTURE')

In [18]:
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_heure)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_heure)
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_heure)
flights['WHEELS_OFF'] = flights['WHEELS_OFF'].apply(format_heure)
flights['WHEELS_ON'] = flights['WHEELS_ON'].apply(format_heure)

In [21]:
flights.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,MONTH_desc,DOW_desc,DATE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,00:15:00,205.0,194.0,169.0,1448,04:04:00,4.0,04:30:00,04:08:00,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,January,Friday,2015-01-01
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,00:14:00,280.0,279.0,263.0,2330,07:37:00,4.0,07:50:00,07:41:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,January,Friday,2015-01-01
2,2015,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,00:34:00,286.0,293.0,266.0,2296,08:00:00,11.0,08:06:00,08:11:00,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,January,Friday,2015-01-01
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,00:30:00,285.0,281.0,258.0,2342,07:48:00,8.0,08:05:00,07:56:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,January,Friday,2015-01-01
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,00:35:00,235.0,215.0,199.0,1448,02:54:00,5.0,03:20:00,02:59:00,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,January,Friday,2015-01-01


In [ ]:
flights.info()

In [ ]:
corr = flights.corr(method='pearson')

Для построения модели для прогнозирования будем использовать опоздание по колонке ARRIVAL_DELAY

In [ ]:
corr['ARRIVAL_DELAY']

In [ ]:
flights.columns

Убираем ненужные колонки

In [22]:
flights = flights.drop(['YEAR', 'DAY','DAY_OF_WEEK', 'FLIGHT_NUMBER','DIVERTED','CANCELLED', 'TAIL_NUMBER', 'AIR_TIME', 'CANCELLATION_REASON', 'DISTANCE'], axis = 1)

Проверяем, есть ли данные с пропущенными значениями

In [23]:
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,WEATHER_DELAY,4755640,18.275040
1,LATE_AIRCRAFT_DELAY,4755640,18.275040
2,AIRLINE_DELAY,4755640,18.275040
3,SECURITY_DELAY,4755640,18.275040
4,AIR_SYSTEM_DELAY,4755640,18.275040
5,ELAPSED_TIME,105071,98.194371
6,ARRIVAL_DELAY,105071,98.194371
7,WHEELS_ON,92513,98.410178
8,TAXI_IN,92513,98.410178
9,ARRIVAL_TIME,92513,98.410178


Удаляем строки с пропущенными значениями (для нужных переменных уровень заполненности начинаеется с 98% это хорошо - не будем заполнять средним значением). По переменным с причинами задержки - пропуски заполним нулевым значением (будем считать, что была иная причина задержки)

Удаляем пустые значения

In [24]:
flights.dropna(subset=['ARRIVAL_DELAY','ELAPSED_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','WHEELS_OFF','TAXI_OUT','DEPARTURE_DELAY','DEPARTURE_TIME'],inplace=True)

In [ ]:
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

Заполняем пропущенные значения нулем

In [25]:
flights = flights.fillna(0)

Найти аэропорт с минимальной задержкой вылета

In [ ]:
a0 = flights.loc[flights['DEPARTURE_DELAY'] == flights['DEPARTURE_DELAY'].min()][['ORIGIN_AIRPORT']]

In [ ]:
AIRPORT_min_delay = airports.loc[airports['IATA_CODE'] == a0['ORIGIN_AIRPORT'].values[0]][['AIRPORT']]

In [ ]:
AIRPORT_min_delay

,AIRPORT
9,Adak Airport


Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

In [ ]:
LA = airports.loc[airports['AIRPORT'] == 'Los Angeles International Airport'][['IATA_CODE']]

In [ ]:
b = flights.loc[flights['DESTINATION_AIRPORT'] == LA['IATA_CODE'].values[0]][['AIRLINE', 'ARRIVAL_DELAY']]

In [ ]:
b1 = b.groupby(['AIRLINE']).mean()
b1

,ARRIVAL_DELAY
AIRLINE,
AA,2.122856
AS,-0.525700
B6,-0.022152
DL,1.398882
F9,10.288235
HA,-4.653317
MQ,18.302469
NK,17.138697
OO,8.202029


Под самой пунктуальной будем понимать авиакомпанию, чье среднее значение самое близкое к нулю (то есть время минимально отклоняется от графика)

In [ ]:
myNumber = 0
m = min(b1['ARRIVAL_DELAY'].values, key=lambda x:abs(x-myNumber))

In [ ]:
ar = b1.loc[b1['ARRIVAL_DELAY'] == m]

In [ ]:
PUNCTUAL_AIRLINE = airlines.loc[airlines['IATA_CODE'] == ar.index[0]][['AIRLINE']]

In [ ]:
PUNCTUAL_AIRLINE

,AIRLINE
4,JetBlue Airways


Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [ ]:
TO = flights[['ORIGIN_AIRPORT', 'TAXI_OUT']]
TI = flights[['DESTINATION_AIRPORT', 'TAXI_IN']]
to = TO.groupby(['ORIGIN_AIRPORT']).mean()
ti = TI.groupby(['DESTINATION_AIRPORT']).mean()

In [ ]:
taxi = to.assign(TAXI_IN = ti.TAXI_IN)
taxi = taxi.assign(TAXI_ALL = taxi.TAXI_OUT + taxi.TAXI_IN)

In [ ]:
taxi

,TAXI_OUT,TAXI_IN,TAXI_ALL
ORIGIN_AIRPORT,,,
ABE,13.325223,3.968635,17.293858
ABI,9.182418,4.003309,13.185727
ABQ,11.948608,5.573155,17.521763
ABR,16.502046,4.679945,21.181991
ABY,12.246849,3.429022,15.675871
...,...,...,...
WRG,7.759547,3.704067,11.463615
WYS,11.903846,4.666667,16.570513
XNA,14.605457,5.102742,19.708200


In [ ]:
max_TAXi = taxi.loc[taxi['TAXI_ALL'] == taxi['TAXI_ALL'].max()]

In [ ]:
max_TAXi_AIRPORT = airports.loc[airports['IATA_CODE'] == max_TAXi.index[0]][['AIRPORT']]

In [ ]:
max_TAXi_AIRPORT

,AIRPORT
182,LaGuardia Airport (Marine Air Terminal)


Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета

In [ ]:
!pip install -U lightautoml

In [27]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import f1_score, log_loss, roc_auc_score, roc_curve, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split
import time
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

In [28]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 3 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 300 # Time in seconds for automl run
TARGET_NAME = 'ARRIVAL_DELAY' # Target column name

In [29]:
import torch
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [30]:
Origin_Airport = np.random.choice(flights['ORIGIN_AIRPORT'].unique()) 
data = flights[flights['ORIGIN_AIRPORT'] == Origin_Airport]

In [ ]:
data

In [31]:
%%time

train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE,  
                                         random_state=RANDOM_STATE)
print('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

Data splitted. Parts sizes: train_data = (17096, 24), test_data = (4274, 24)
CPU times: user 22.1 ms, sys: 0 ns, total: 22.1 ms
Wall time: 22.9 ms


In [33]:
task = Task('reg', ) 
            #loss = 'mae', metric = 'mae')

In [57]:
roles = {'target': TARGET_NAME, 'drop' : ['SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DEPARTURE_TIME', 'ARRIVAL_TIME','WHEELS_ON','SCHEDULED_ARRIVAL', 'WHEELS_OFF', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY' ]}

In [58]:
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

In [ ]:
%%time 
oof_pred = automl.fit_predict(train_data, roles = roles, verbose = 1)

In [60]:
print(automl.create_model_str_desc())

Final prediction for new objects (level 0) = 
	 0.69059 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.12822 * (3 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.12735 * (3 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.05383 * (3 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 


In [61]:
%%time

test_pred = automl.predict(test_data)
print(f'Prediction for te_data:\n{test_pred}\nShape = {test_pred.shape}')

Prediction for te_data:
array([[ 42.37812   ],
       [ 10.250923  ],
       [ -5.578416  ],
       ...,
       [  0.44313717],
       [ 11.217718  ],
       [-11.150027  ]], dtype=float32)
Shape = (4274, 1)
CPU times: user 1.06 s, sys: 11.8 ms, total: 1.07 s
Wall time: 728 ms


In [62]:
from sklearn.metrics import mean_absolute_error
from numpy import sqrt 
print(f'TRAIN out-of-fold score: {mean_absolute_error(train_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'HOLDOUT score: {mean_absolute_error(test_data[TARGET_NAME].values, test_pred.data[:, 0])}')

TRAIN out-of-fold score: 19.13399058540992
HOLDOUT score: 19.432858820445567


In [63]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(np.array(test_pred.data[:, 0]), np.array(test_data[TARGET_NAME].values))
print("rms error is: " + str(rmse_val))

rms error is: 40.44407919931397


In [64]:
from sklearn.metrics import r2_score
print(f'R^2 train: {r2_score(train_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'R^2 test: {r2_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])}')

R^2 train: 0.13765056295428402
R^2 test: 0.12571665306685342


In [65]:
from sklearn.metrics import mean_squared_error
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)
print('Check scores:')
print('OOF score: {}'.format(sqrt(mean_squared_error(train_data[TARGET_NAME].values[not_nan], oof_pred.data[not_nan][:, 0]))))
print('TEST score: {}'.format(sqrt(mean_squared_error(test_data[TARGET_NAME].values,test_pred.data[:, 0]))))

Check scores:
OOF score: 40.77816264531203
TEST score: 40.44407919931397


In [66]:
td = test_data[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ARRIVAL_DELAY']]
df = pd.DataFrame(test_pred.data[:, 0], columns = ['Prediction_ARRIVAL_DELAY'])

In [67]:
td.index = np.arange(0,len(td))

In [68]:
a = pd.concat([td,df], sort=False, axis=1)
a.head()

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,ARRIVAL_DELAY,Prediction_ARRIVAL_DELAY
0,CVG,IAH,37.0,42.378120
1,CVG,ATL,-3.0,10.250923
2,CVG,XNA,-9.0,-5.578416
3,CVG,ATL,-8.0,-1.468822
4,CVG,ORD,4.0,17.285606


In [69]:
final = []

In [70]:
for uniq_aiport in a['DESTINATION_AIRPORT'].unique():
  rmse = np.sqrt(mean_squared_error(a.ARRIVAL_DELAY[a['DESTINATION_AIRPORT'] == uniq_aiport], a.Prediction_ARRIVAL_DELAY[a['DESTINATION_AIRPORT'] == uniq_aiport]))
  final.append([Origin_Airport, uniq_aiport, rmse])


In [71]:
F = pd.DataFrame(final)
F.columns = ['Arrival_Airport', 'Destination_Airport', 'RMSE']

In [72]:
F = F.sort_values(by=['RMSE'])

In [73]:
F.head(3)

,Arrival_Airport,Destination_Airport,RMSE
36,CVG,PIT,10.105065
33,CVG,MCI,11.597909
29,CVG,GRR,12.715458
